# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [27]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图显示了一个美丽的自然景观，是一个宁静的湿地环境。图中心是一条木制的栈道，引导观者的视线深入画面，通向远处的地平线。四周被翠绿的草地和杂树丛环绕，这为画面增添了丰富的绿色调。这个场景可能位于一个自然保护区或公园内，旨在保护地面不受过度行走的破坏，同时提供给游人一条安全的观赏路线。\n\n天空呈现出动人的蓝色和白色云朵，增强了画面的深度和广阔感。整体上，这幅图展示了大自然的宁静与和谐，是自然美景与人类设施协调共存的一个例证。这样的环境是很适合散步、放松和自然观察的地方。', role='assistant', function_call=None, tool_calls=None))


In [28]:
response.choices[0].message.content

'这幅图显示了一个美丽的自然景观，是一个宁静的湿地环境。图中心是一条木制的栈道，引导观者的视线深入画面，通向远处的地平线。四周被翠绿的草地和杂树丛环绕，这为画面增添了丰富的绿色调。这个场景可能位于一个自然保护区或公园内，旨在保护地面不受过度行走的破坏，同时提供给游人一条安全的观赏路线。\n\n天空呈现出动人的蓝色和白色云朵，增强了画面的深度和广阔感。整体上，这幅图展示了大自然的宁静与和谐，是自然美景与人类设施协调共存的一个例证。这样的环境是很适合散步、放松和自然观察的地方。'

### 封装成一个函数 query_image_description

In [29]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [30]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图描绘了两种不同表达方式的狗的卡通形象。左边的图显示了一只肌肉发达的狗，配文字“16岁的我，工作后的我”，意在幽默地展示年轻时的活力和强健。它身上的文字描述了各种积极和充满活力的特点，如“我刚像一片光明、大家阿姨、少年气盛”等。

右边的图则表现了一只看起来比较悲观和疲惫的狗，文字描述反映了工作生活中可能遇到的压力和挑战，例如“好累好累，好想摸摸你” 和 “工友不理我就我摸我自己”。

总体上，这幅图通过对比展示了一个人工作前后可能经历的变化，用幽默的方式表达了工作生活可能带来的压力与消极情绪。这种幽默和夸张的表达方式


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [31]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)
    print(response)
    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [32]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

<Response [200]>
这幅图显示了1980年到2020年期间，美国、中国、日本和德国的国内生产总值（GDP）的比较。其中，各国的GDP用不同颜色的线表示：

- 蓝线代表美国
- 红线代表中国
- 紫线代表日本
- 绿线代表德国

从图中可以看出，美国的GDP在这40年间一直呈现上升趋势，且在各国中排名第一。中国的GDP增长非常显著，从图中可以看到自1990年代初以来，其增速加快，到2010年代超过了日本，成为第二大经济体。日本的GDP在1990年代初达到峰值后略有波动，之后基本稳定。德国的GDP增长较为平稳，位于日本之后。

总的来说，这个图表提供了一个清晰的视角来比较这四个国家经济的总体规模和增长趋势。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [33]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

<Response [200]>
这张图片中展示的内容是一本笔记本上的笔记，主题涉及人工智能领域中的几种参数调整技术，如prompt tuning、prefix tuning以及LoRA。

1. **Prompt Tuning（提示调整）**：这种技术涉及使用小模型，并与输入数据一起创建输入嵌入，改变模型在处理数据时的行为模式。

2. **Prefix Tuning（前缀调整）**：在这种方法中，会给模型的输入添加前缀，通过引入W'来影响输入X，并通过Transformer的编码器/解码器对这些数据进行处理。

3. **LoRA（低秩适应）**：这部分笔记详细说明了如何通过调整权重矩阵W来引入一个小的变化ΔW，来获得新的输出Y。这种方法通过减小参数的更改来节省存储空间和计算资源，可以看到笔记中提到的LoRA和QLoRA，以及相关的存储数据需求（如LoRA - 78GB和QLoRA - 48GB）。

整体来看，这些笔记可能是在研读如何使用这些先进的调整技术来改进或微调深度学习模型，特别是在自然语言处理或类似领域中。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [34]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片中展示的内容是一本笔记本上的笔记，主题涉及人工智能领域中的几种参数调整技术，如prompt tuning、prefix tuning以及LoRA。

1. **Prompt Tuning（提示调整）**：这种技术涉及使用小模型，并与输入数据一起创建输入嵌入，改变模型在处理数据时的行为模式。

2. **Prefix Tuning（前缀调整）**：在这种方法中，会给模型的输入添加前缀，通过引入W'来影响输入X，并通过Transformer的编码器/解码器对这些数据进行处理。

3. **LoRA（低秩适应）**：这部分笔记详细说明了如何通过调整权重矩阵W来引入一个小的变化ΔW，来获得新的输出Y。这种方法通过减小参数的更改来节省存储空间和计算资源，可以看到笔记中提到的LoRA和QLoRA，以及相关的存储数据需求（如LoRA - 78GB和QLoRA - 48GB）。

整体来看，这些笔记可能是在研读如何使用这些先进的调整技术来改进或微调深度学习模型，特别是在自然语言处理或类似领域中。

![](./images/handwriting_1.jpg)

In [35]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

<Response [200]>


这张图片显示的是一本笔记本的两页，内容主要涉及人工智能中的“变换器(transformers)”模型，特别是在自然语言处理(NLP)领域中的各种调优技术和方法。左边的部分主要描述了与变换器相关的一些方法如：PEFT、SOTA、PBFT Methods 以及各种Prompt Tuning技术，例如Adapter、Prefix、P-tuning等。右边的部分讨论了多模态指导微调、LoRA、PETC等新方法，并提到了关于Adapters的一些细节和优化，例如MAM Adapters和对比函数等。

这些笔记看上去是某位研究人员或学生在学习或整理关于最新的AI模型调整技术时的记录。笔记详细记录了不同技术的发表时间、研究机构、核心概念以及它们在语言模型中的应用，这对于追踪技术发展或进行科学研究都很有帮助。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [37]:
# 使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果
content = query_base64_image_description("./images/gpt4-v.jpg")
print(content)

<Response [200]>
这张图片展示的是关于检查放行订单流程的笔记。详细步骤如下：

1. **检查提单(B/L)是否设置了放行订单**
   - 提单（B/L）是指货物运输中的提货单或者装货单，这一步检查是否进行了放行订单的设置。

2. **检查提单是否有放行订单的接收者**
   - 这一步检查的是是否已经指定了放行订单的接收方。

3. **检查提单货物的放行状态是否全部清除**
   - 检查提单上的货物是否已经获得了全部必要的清关放行，以使其能够被提取。

4. **检查提单的PIN状态，如果提单的最后转运中心是Antwerp, 接着检查终端放行状态**
   - 这里涉及到用PIN码进行身份验证或安全检查，特别是如果货物的最后转运中心是Antwerp时，还需要检查货物在终端是否被放行。

这些笔记可能是用于受托管理货物运输和放行的操作步骤，确保各个环节按照规定进行，以避免在运输或接收过程中出现问题。


In [38]:
# 整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。
from IPython.display import display, Markdown
content = query_base64_image_description("./images/gpt4-v.jpg")
display(Markdown(content))

<Response [200]>


这张图片显示的是一份手写的笔记，内容关于检查释放订单的流程。具体步骤如下：

1. Check BL has release order setting - 检查提单（BL，Bill of Lading）是否设置了释放订单。
2. Check BL has Release order Recipient - 检查提单是否有释放订单的接收者。
3. Check BL cargo release status all cleared - 检查提单的货物释放状态是否全部清除。
4. Check BL PIN status, if BL Last hub is Antwerp, then check terminal release status. - 检查提单的PIN状态，如果提单的最后一个枢纽是安特卫普，那么检查终端的释放状态。

这些步骤是为了确保货物的分派与接收在物流过程中是清晰且有序的，特别是在不同的地点和可能的法律条款之间。